In [1]:
from graphviz import Digraph
import itertools

In [2]:
def d_relations(A, I):
    return [relation for relation in sorted(itertools.product(A, A)) if relation not in I]

In [3]:
def trace(w):
    return "[" + w + "]"
#trace - all posibilities form from dependency graph(string permutation), simples just to write w

In [4]:
def foata_normal_form(A, I, w):
    result = []
    # each letter is independent item, abcda - 1st and last 'a' - indepented item
    # in result we have pair item + index in w
    stacks = generate_stacks(A, I, w)
    top_stack_size = max([len(stack) for stack in stacks.values()])

    for i in range(top_stack_size):
        popped = []
        for letter in stacks:
            stack = stacks[letter]
            if len(stack) > 0 and stack[-1] != '*':
                popped_letter = stack.pop()
                popped.append(popped_letter)
        
        #now pop '*' for stacks with letter not in I with popped letters
        if not popped:
            for letter in stacks:
                if len(stacks[letter]) > 0:
                    stacks[letter].pop()
        else:
            for letter in A:
                popped_letter = popped[0]
                if letter != popped_letter and (letter, popped_letter) not in I and len(stacks[letter]) > 0 and stacks[letter][-1] == '*':
                    a = stacks[letter].pop()
            result.append(sorted(popped))
        
    output = ''
    for group in result:
        output += "(" + "".join([i[0] for i in group]) + ")"
    print(output)
    return result


def generate_stacks(A, I, w):
    stacks = {letter: [] for letter in A}
    for i, a in enumerate(w[::-1]):
        stacks[a].append((a, str(w.index(a, len(w)-i-1) + 1)))
        for b in A:
            if b != a and (a, b) not in I:
                stacks[b].append("*")
    return stacks

In [5]:
def dependency_graph(normal_form, I):
    dot = Digraph()

    #Labels for nodes
    for group in normal_form:
        for node in group:
            dot.node(node[1], node[0])

    # to check if two nodes are connected indirectly
    edges = []
    result = []
    #we go from right to left in foata normal form
    for i in range(len(normal_form) - 2, -1, -1): 
        for u in normal_form[i]: # node
            for j in range(i + 1, len(normal_form)):  #i+1, we start with normal_form[-1] so we don't get our of list 
                for v in normal_form[j]:  # second node
                    #we cant have edges a-a, or b-a-d, and b connected with d, we dont want edge b - d
                    if (u[0], v[0]) not in I and (u, v) not in edges:
                        edges.append((u, v))
                        for edge in edges:
                            if edge[0] == v:
                                edges.append((u, edge[1]))
                        dot.edge(u[1], v[1])
                        result.append((u[0], v[0]))
    print(dot.source)
    return result

In [6]:
def foata_from_graph(graph, A, w):
    I = get_I_from_graps(graph, A)
#     I = [('a', 'd'), ('d', 'a'), ('b', 'c'), ('c', 'b')]
    print("Foata form from graph:")
    foata_normal_form(A, I, w)

def get_I_from_graps(graph, A):
    I_compl = {letter: set(letter) for letter in A}
    for edge in graph:
        u = edge[0][0]
        v = edge[1][0]
        I_compl[u].add(v)
        I_compl[v].add(u)
    I = set()
    for key in I_compl:
        for letter in A:
            if letter not in I_compl[key]:
                I.add((key, letter))
                I.add((letter, key))
    return I

In [7]:
def execute(A, I, w):
    print("D = ", d_relations(A, I))
    print("[w]: ", trace(w))
    normal_form = foata_normal_form(A, I, w)
    graph = dependency_graph(normal_form, I)
    foata_from_graph(graph, A, w)

## Example 1

In [8]:
A = {'a', 'b', 'c', 'd'}
I = [('a', 'd'), ('d', 'a'), ('b', 'c'), ('c', 'b')]
w = 'baadcb'
execute(A, I, w)

D =  [('a', 'a'), ('a', 'b'), ('a', 'c'), ('b', 'a'), ('b', 'b'), ('b', 'd'), ('c', 'a'), ('c', 'c'), ('c', 'd'), ('d', 'b'), ('d', 'c'), ('d', 'd')]
[w]:  [baadcb]
(b)(ad)(a)(bc)
digraph {
	1 [label=b]
	2 [label=a]
	4 [label=d]
	3 [label=a]
	6 [label=b]
	5 [label=c]
	3 -> 6
	3 -> 5
	2 -> 3
	4 -> 6
	4 -> 5
	1 -> 2
	1 -> 4
}
Foata form from graph:
(b)(ad)(a)(bc)


## Example 2

In [9]:
A = {'a', 'b', 'c', 'd', 'e', 'f'}
I = {('a', 'd'), ('d', 'a'), ('b', 'e'), ('e', 'b'), ('c', 'd'), ('d', 'c'), ('c', 'f'), ('f', 'c')}
w = 'acdcfbbe'
execute(A, I, w)

D =  [('a', 'a'), ('a', 'b'), ('a', 'c'), ('a', 'e'), ('a', 'f'), ('b', 'a'), ('b', 'b'), ('b', 'c'), ('b', 'd'), ('b', 'f'), ('c', 'a'), ('c', 'b'), ('c', 'c'), ('c', 'e'), ('d', 'b'), ('d', 'd'), ('d', 'e'), ('d', 'f'), ('e', 'a'), ('e', 'c'), ('e', 'd'), ('e', 'e'), ('e', 'f'), ('f', 'a'), ('f', 'b'), ('f', 'd'), ('f', 'e'), ('f', 'f')]
[w]:  [acdcfbbe]
(ad)(c)(cf)(be)(b)
digraph {
	1 [label=a]
	3 [label=d]
	2 [label=c]
	4 [label=c]
	5 [label=f]
	6 [label=b]
	8 [label=e]
	7 [label=b]
	6 -> 7
	4 -> 6
	4 -> 8
	5 -> 6
	5 -> 8
	2 -> 4
	1 -> 2
	1 -> 5
	3 -> 5
}
Foata form from graph:
(ad)(c)(cf)(be)(b)


## Example 3

In [10]:
A = {'a', 'b', 'c', 'd', 'e'}
I = {('b', 'd'), ('d', 'b'), ('b', 'e'), ('e', 'b')}
w = 'acebdac'
execute(A, I, w)

D =  [('a', 'a'), ('a', 'b'), ('a', 'c'), ('a', 'd'), ('a', 'e'), ('b', 'a'), ('b', 'b'), ('b', 'c'), ('c', 'a'), ('c', 'b'), ('c', 'c'), ('c', 'd'), ('c', 'e'), ('d', 'a'), ('d', 'c'), ('d', 'd'), ('d', 'e'), ('e', 'a'), ('e', 'c'), ('e', 'd'), ('e', 'e')]
[w]:  [acebdac]
(a)(c)(be)(d)(a)(c)
digraph {
	1 [label=a]
	2 [label=c]
	4 [label=b]
	3 [label=e]
	5 [label=d]
	6 [label=a]
	7 [label=c]
	6 -> 7
	5 -> 6
	4 -> 6
	3 -> 5
	2 -> 4
	2 -> 3
	1 -> 2
}
Foata form from graph:
(a)(ce)(bd)(a)(c)
